# Request Prime Counter



In [1]:
import numpy as np
import pandas as pd

## - Read samples.txt file

In [2]:
# read file
data = pd.read_csv('sample_data/training.txt', sep=" ", header=None, dtype=str)
data = data.astype(float)
# print(data)
data[[2, 3]] = data[[2, 3]] * (10 ** 2)
data = data.dropna(axis=1)
print(data)

            0      1          2           3           4    5         6  \
0       971.0  164.0  16.139148  100.000000    2.160820  1.0  0.488348   
1      2686.0  389.0  15.916885  100.000000    6.080928  0.0  0.451309   
2      6203.0  807.0  16.201186  100.000000    8.224186  1.0  0.488348   
3      3775.0  525.0  15.969617   91.733985   11.474194  0.0  0.451309   
4      7712.0  978.0  15.961208  100.000000   83.647613  1.0  0.488348   
...       ...    ...        ...         ...         ...  ...       ...   
49995  4210.0  575.0  16.306502  100.000000    5.118050  1.0  0.475488   
49996  3938.0  546.0  15.802902  100.000000  101.251037  0.0  0.441206   
49997  1581.0  249.0  15.802902   30.613012  103.656416  0.0  0.441206   
49998  4596.0  621.0  16.315808  100.000000   64.993985  1.0  0.475455   
49999  4134.0  569.0  16.315808   98.026474   77.640803  1.0  0.475446   

              7  
0      0.156381  
1      0.159841  
2      0.156381  
3      0.159841  
4      0.156381  
...

## - Label data for features and taget variable

In [3]:
# Split X and Y
X = data.drop(data.columns[[1, 2, 6, 7]], axis=1)
Y = data.iloc[:, [4]]


print(X)
print(Y)

            0           3           4    5
0       971.0  100.000000    2.160820  1.0
1      2686.0  100.000000    6.080928  0.0
2      6203.0  100.000000    8.224186  1.0
3      3775.0   91.733985   11.474194  0.0
4      7712.0  100.000000   83.647613  1.0
...       ...         ...         ...  ...
49995  4210.0  100.000000    5.118050  1.0
49996  3938.0  100.000000  101.251037  0.0
49997  1581.0   30.613012  103.656416  0.0
49998  4596.0  100.000000   64.993985  1.0
49999  4134.0   98.026474   77.640803  1.0

[50000 rows x 4 columns]
                4
0        2.160820
1        6.080928
2        8.224186
3       11.474194
4       83.647613
...           ...
49995    5.118050
49996  101.251037
49997  103.656416
49998   64.993985
49999   77.640803

[50000 rows x 1 columns]


## - Common scale

In [4]:
from sklearn.preprocessing import StandardScaler, RobustScaler

robust_scaler = RobustScaler()
X_robust = robust_scaler.fit_transform(X)

## - Split data into training set validation set and testing set

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

# Build MLP Model

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from tensorflow.keras.optimizers import Adam

# Convert DataFrames to NumPy arrays and reshape the input data for a 1D CNN
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)

# Ensure Y labels are shaped correctly for the model
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_val = np.array(Y_val)

# Build the model with a 1D CNN
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train.shape[1], 1)),
    # Dense(64, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model with the mean squared error (MSE) loss function
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])

# Train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

# Evaluate the model on the test set
loss, mse = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss:.5f}')
print(f'Test Mean Squared Error: {mse:.5f}')


Epoch 1/10
1250/1250 [==============================] - 8s 5ms/step - loss: 91.1753 - mse: 91.1753 - val_loss: 8.6826 - val_mse: 8.6826
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 14.8636 - mse: 14.8636 - val_loss: 3.7211 - val_mse: 3.7211
Epoch 3/10
1250/1250 [==============================] - 7s 6ms/step - loss: 2.6629 - mse: 2.6629 - val_loss: 1.5135 - val_mse: 1.5135
Epoch 4/10
1250/1250 [==============================] - 6s 5ms/step - loss: 8.0910 - mse: 8.0910 - val_loss: 0.7011 - val_mse: 0.7011
Epoch 5/10
1250/1250 [==============================] - 8s 6ms/step - loss: 0.9670 - mse: 0.9670 - val_loss: 1.0652 - val_mse: 1.0652
Epoch 6/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.9054 - mse: 0.9054 - val_loss: 2.9487 - val_mse: 2.9487
Epoch 7/10
1250/1250 [==============================] - 7s 6ms/step - loss: 3.5695 - mse: 3.5695 - val_loss: 0.3148 - val_mse: 0.3148
Epoch 8/10
1250/1250 [==============================] - 5s

In [11]:
model.save('LSTM_mlp_time_pridection_model.keras')